# Shallow Moment Tutorial (Minimal)

## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np
import jax
from jax import numpy as jnp
import pytest
from types import SimpleNamespace
from sympy import cos, pi, Piecewise

from library.fvm.solver import HyperbolicSolver
from library.fvm.ode import RK1
import library.fvm.reconstruction as recon
import library.fvm.timestepping as timestepping
import library.fvm.flux as flux
import library.fvm.nonconservative_flux as nc_flux
from library.model.boundary_conditions import BoundaryCondition
from library.model.models.basisfunctions import Basisfunction, Legendre_shifted
from library.model.models.basismatrices import Basismatrices
from library.model.models.shallow_moments import ShallowMoments2d
import library.misc.io as io

import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
import library.misc.io as io
from library.mesh.mesh import compute_derivatives
from tests.pdesoft import plots_paper

from library.misc.misc import Zstruct, Settings


import library.mesh.mesh as petscMesh
import library.postprocessing.postprocessing as postprocessing
from library.mesh.mesh import convert_mesh_to_jax
import argparse


2025-08-07 07:56:45.399 | WARNING  | library.fvm.solver:<module>:24 - No module named 'precice'


## Model

In [2]:
level = 0
offset = 1 + level
settings = Settings(
    name="test",
    model=Zstruct(
        level=level,
        parameters=Zstruct(
            g=9.81, ex=0, ey=0, ez=1, gz=0, rho=1, nu=0.000001, lamda=0.1
        ),
    ),
    solver=Zstruct(time_end=0.1, compute_dt=timestepping.adaptive(CFL=0.45)),
    output=Zstruct(
        directory=f"outputs/junction_{level}", filename="test", output_snapshots=30
    ),
)


2025-08-07 07:56:48.393 | WARNING  | library.misc.misc:__init__:153 - No 'clean_directory' attribute found in output Zstruct. Default: False


In [3]:
inflow_dict = {
    0: lambda t, x, dx, q, qaux, p, n: Piecewise((0.1, t < 0.2), (q[0], True)),
    1: lambda t, x, dx, q, qaux, p, n: Piecewise((-0.3, t < 0.2), (-q[1], True)),
}
inflow_dict.update({1 + i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level)})
inflow_dict.update(
    {1 + offset + i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level + 1)}
)

bcs = BC.BoundaryConditions(
    [
        BC.Lambda(physical_tag="inflow", prescribe_fields=inflow_dict),
        BC.Wall(physical_tag="wall"),
    ]
)


def custom_ic(x):
    Q = np.zeros(3 + 2 * level, dtype=float)
    Q[0] = 0.01
    return Q


ic = IC.UserFunction(custom_ic)

model = ShallowMoments2d(
    fields=3 + 2 * level,
    aux_fields=2,
    parameters=settings.model.parameters.as_dict(),
    boundary_conditions=bcs,
    initial_conditions=ic,
    settings={"friction": ["newtonian", "slip_mod"]},
    basis=Basismatrices(basis=Legendre_shifted(order=level + 1)),
)

main_dir = os.getenv("SMS")
mesh = petscMesh.Mesh.from_gmsh(
    os.path.join(main_dir, "meshes/channel_junction/mesh_2d_coarse.msh")
    # os.path.join(main_dir, "meshes/channel_junction/mesh_2d_fine.msh")
)


# mesh = convert_mesh_to_jax(mesh)
class SMESolver(HyperbolicSolver):
    def update_qaux(self, Q, Qaux, Qold, Qauxold, mesh, model, parameters, time, dt):
        dudx = compute_derivatives(Q[1] / Q[0], mesh, derivatives_multi_index=[[0, 0]])[
            :, 0
        ]
        dvdy = compute_derivatives(
            Q[1 + offset] / Q[0], mesh, derivatives_multi_index=[[0, 1]]
        )[:, 0]
        Qaux = Qaux.at[0].set(dudx)
        Qaux = Qaux.at[1].set(dvdy)
        return Qaux


solver = SMESolver(settings)


## Solver

In [4]:
Qnew, Qaux = solver.solve(mesh, model)

io.generate_vtk(
    os.path.join(settings.output.directory, f"{settings.output.filename}.h5")
)

2025-08-07 07:57:09.493 | WARNING  | library.misc.io:write_dict_to_hdf5:51 - Skipping unsupported type for key: num_flux -> <class 'function'>
2025-08-07 07:57:09.494 | WARNING  | library.misc.io:write_dict_to_hdf5:51 - Skipping unsupported type for key: nc_flux -> <class 'function'>
2025-08-07 07:57:09.496 | WARNING  | library.misc.io:write_dict_to_hdf5:51 - Skipping unsupported type for key: compute_dt -> <class 'function'>
2025-08-07 07:57:13.535 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 1, time: 0.006773, dt: 0.006773, next write at time: 0.011111
2025-08-07 07:57:13.600 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 2, time: 0.007304, dt: 0.000532, next write at time: 0.011111
2025-08-07 07:57:13.664 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 3, time: 0.007836, dt: 0.000532, next write at time: 0.011111
2025-08-07 07:57:13.750 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 

## Visualization

In [5]:
io.generate_vtk(os.path.join(settings.output.directory, f"{settings.name}.h5"))
postprocessing.vtk_interpolate_3d(model, settings, Nz=20, filename='out_3d')

2025-08-07 07:57:48.420 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:63 - Converted snapshot 0/10
2025-08-07 07:57:48.491 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:63 - Converted snapshot 1/10
2025-08-07 07:57:48.598 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:63 - Converted snapshot 2/10
2025-08-07 07:57:48.714 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:63 - Converted snapshot 3/10
2025-08-07 07:57:48.794 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:63 - Converted snapshot 4/10
2025-08-07 07:57:48.872 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:63 - Converted snapshot 5/10
2025-08-07 07:57:48.957 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:63 - Converted snapshot 6/10
2025-08-07 07:57:49.046 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:63 - Converted snapshot 7/10
2025-08-07 07:57